# Text Classification Using ALBERT
 Fine Tuning Transformer for MultiLabel Text Classification

Installing Transformers

In [20]:
!pip install -q transformers

In [21]:
# # Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

## Importing Necessary Libraries

In [40]:
# Importing stock ml libraries
import numpy as np
import pandas as pd
from sklearn import metrics
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import transformers
import requests
from io import BytesIO
import tarfile

from transformers import AlbertTokenizer, AlbertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report




## Extract Data from source

In [7]:
url = "https://s3.amazonaws.com/fast-ai-nlp/amazon_review_full_csv.tgz"
response = requests.get(url)
file = tarfile.open(fileobj=BytesIO(response.content))
file.extractall()

In [23]:
train = pd.read_csv('amazon_review_full_csv/train.csv',usecols=[0,1,2], names=['Rating', 'Title','ReviewText'],nrows=10000)
test = pd.read_csv('amazon_review_full_csv/test.csv',usecols=[0,1,2], names=['Rating', 'Title','ReviewText'],nrows=3500)

In [24]:
train.shape

(10000, 3)

In [25]:
test.shape

(3500, 3)

In [26]:
# Define function to classify ratings
def classify_rating(rating):
    if rating > 3:
        return 'Positive'
    elif rating < 3:
        return 'Negative'
    else:
        return 'Neutral'

In [27]:
train['Sentiment'] = train['Rating'].apply(lambda x: classify_rating(x))
test['Sentiment'] = test['Rating'].apply(lambda x: classify_rating(x))

In [28]:
train,test = train[['Sentiment','ReviewText']], test[['Sentiment','ReviewText']]

## Convert labels to numeric values

In [29]:

label2id = {"Positive": 2, "Neutral": 1, "Negative": 0}
train["label"] = train["Sentiment"].map(label2id)
test["label"] = test["Sentiment"].map(label2id)

In [30]:
train["Sentiment"] = train["label"]
test["Sentiment"] = test["label"]

In [31]:
train,test = train[['Sentiment','ReviewText']], test[['Sentiment','ReviewText']]

In [32]:
train.head()

,Sentiment,ReviewText
0,1,Gave this to my dad for a gag gift after direc...
1,2,I hope a lot of people hear this cd. We need m...
2,2,I'm reading a lot of reviews saying that this ...
3,2,The music of Yasunori Misuda is without questi...
4,2,Probably the greatest soundtrack in history! U...


In [33]:
test.head()

,Sentiment,ReviewText
0,0,"This model may be ok for sedentary types, but ..."
1,2,This is a fast read filled with unexpected hum...
2,0,I bought one of these chargers..the instructio...
3,0,I was excited to find a book ostensibly about ...
4,0,"I am a big JVC fan, but I do not like this mod..."


## Load the BERT tokenizer and encode the text:

In [34]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [36]:
class AmazonReviewDataset(Dataset):
    def __init__(self, reviews, labels):
        self.reviews = reviews
        self.Sentiment = labels

    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, idx):
        review = str(self.reviews[idx])
        label = self.Sentiment[idx]
        encoding = tokenizer.encode_plus(
            review,
            add_special_tokens=True,
            max_length=512,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        return {
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
            "label": torch.tensor(label, dtype=torch.long)
        }



In [37]:
train_dataset = AmazonReviewDataset(train["ReviewText"], train["Sentiment"])
val_dataset = AmazonReviewDataset(test["ReviewText"], test["Sentiment"])

## Create data loaders:

In [38]:
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)


## Load the pre-trained BERT model for sequence classification:

In [41]:
model = AlbertForSequenceClassification.from_pretrained("albert-base-v2", num_labels=len(label2id))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForSequenceClassification: ['predictions.decoder.weight', 'predictions.decoder.bias', 'predictions.dense.bias', 'predictions.bias', 'predictions.LayerNorm.weight', 'predictions.dense.weight', 'predictions.LayerNorm.bias']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You sho

AlbertForSequenceClassification(
  (albert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(30000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768,

## Define the optimizer and learning rate scheduler:

In [42]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
total_steps = len(train_loader) * 5  # 5 epochs
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)


## Train the model:

In [43]:
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{num_epochs} - Avg Loss: {avg_loss:.4f}")

    # Validation
    model.eval()
    val_predictions = []
    val_labels = []
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            _, predictions = torch.max(logits, dim=1)
            val_predictions.extend(predictions.detach().cpu().numpy())
            val_labels.extend(labels.detach().cpu().numpy())
    val_acc = accuracy_score(val_labels, val_predictions)
    print(f"Validation Accuracy: {val_acc:.4f}")


Epoch 1/5 - Avg Loss: 1.0637
Validation Accuracy: 0.4117
Epoch 2/5 - Avg Loss: 1.0641
Validation Accuracy: 0.4117
Epoch 3/5 - Avg Loss: 1.0652
Validation Accuracy: 0.4117
Epoch 4/5 - Avg Loss: 1.0646
Validation Accuracy: 0.4117
Epoch 5/5 - Avg Loss: 1.0626
Validation Accuracy: 0.4117


## Evaluate the model on training set

In [44]:
# Evaluation on the train set

model.eval()
train_predictions = []
train_labels = []
with torch.no_grad():
    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        _, predictions = torch.max(logits, dim=1)
        train_predictions.extend(predictions.detach().cpu().numpy())
        train_labels.extend(labels.detach().cpu().numpy())

train_acc = accuracy_score(train_labels, train_predictions)
classification_rep = classification_report(train_labels, train_predictions, target_names=label2id.keys())

print(f"Train Accuracy: {train_acc:.4f}")
print("\nClassification Report:")
print(classification_rep)



Train Accuracy: 0.4135

Classification Report:
              precision    recall  f1-score   support

    Positive       0.42      0.57      0.48      4073
     Neutral       0.17      0.00      0.00      2010
    Negative       0.41      0.46      0.43      3917

    accuracy                           0.41     10000
   macro avg       0.33      0.34      0.31     10000
weighted avg       0.37      0.41      0.37     10000

